# getting-started-trello-api

## reference

* [Rate Limits](https://help.trello.com/article/838-api-rate-limits): **100 calls per 10 sec** for each key/token
* [Batch requests](https://developer.atlassian.com/cloud/trello/rest/api-group-batch/)
* [Search](https://developer.atlassian.com/cloud/trello/rest/api-group-search/#api-search-get) (query: `is:archived`)
* [Delete cards](https://developer.atlassian.com/cloud/trello/rest/api-group-cards/#api-cards-id-delete)

## setup

In [1]:
from pathlib import Path
import json
import time

import grequests
import requests

path_trello_creds = Path('/home/selim/.trello')

## get trello credentials.
## should be in ~/.trello, with the following format:
#key=...
#token=...
with open(path_trello_creds, 'r') as f:
    c = f.read()
creds_key = c.split('\n')[0].split('=')[1]
creds_token = c.split('\n')[1].split('=')[1]

## API CALL

### get archived cards

In [2]:
def get_archived_cards(key, token, query='is:archived', cards_limit=1000, verbose=1):
    if verbose > 0:
        print('getting list of archived cards...')
    params = {'key':creds_key, 'token':creds_token, 'query':query, 'cards_limit':cards_limit}
    url_search = 'https://api.trello.com/1/search'
    r = requests.get(url_search, params=params)
    if verbose > 0:
        print(r)
    out = json.loads(r.text)
    return(out['cards'])

### delete

### batch requests

trello batch requests only work for GET

In [6]:
total_cards_processed = 0
nb_batch = 100
sleep_sec = 10
params = {'key':creds_key, 'token':creds_token}
logs = list()
cards = get_archived_cards(creds_key, creds_token)
while len(cards) > 0:
    print(f'found {len(cards)} cards archived')
    cards_id = [c['id'] for c in cards]
    cards_name = [c['name'] for c in cards]
    nb_batchs = (len(cards) // nb_batch) + 1*(len(cards)%nb_batch > 0)
    print(f'nb of batches: {nb_batchs}')
    
    for i in range(nb_batchs):
        tmp_ids = cards_id[i*nb_batch:min((i+1)*nb_batch, len(cards))]
        tmp_names = cards_name[i*nb_batch:min((i+1)*nb_batch, len(cards))]
        urls = [f'https://api.trello.com/1/cards/{cid}' for cid in tmp_ids]
        async_list = [grequests.request('DELETE', u, params=params) for u in urls]
        print(f'running {len(async_list)} requests asynchronously')
        outputs = grequests.map(async_list)
        total_cards_processed += len(cards)
        for (cid, name, r) in zip(tmp_ids, tmp_names, outputs):
            logs.append({'id': cid, 'name': name, 'response':r})
        print(f'sleeping for {sleep_sec} seconds...')
        time.sleep(sleep_sec)

    print(f'updating list of archived cards...')
    cards = get_archived_cards(creds_key, creds_token)
    time.sleep(sleep_sec)

getting list of archived cards...
<Response [200]>
found 1000 cards archived
nb of batches: 10
running 100 requests asynchronously
sleeping for 10 seconds...


KeyboardInterrupt
2020-08-10T13:44:55Z


KeyboardInterrupt: 

In [5]:
#assert len(logs) == total_cards_processed
nb_success = len([l for l in logs if l['response'].status_code == 200]) 
errors = [l for l in logs if l['response'].status_code != 200]

print(f'total nb of cards processed: {total_cards_processed}')
print(f'successful processing: {nb_success}/{total_cards_processed}')

total nb of cards processed: 2264
successful processing: 2463/2264
